# HTTP 伺服器

執行 HTTP 伺服器，在收到 GET 請求時向 Kachaka 發送指令的範例。
透過從 Kachaka 按鈕 Hub 等裝置向以下地址發送 GET 請求，即可發送所需的指令。
```
http://<Kachaka的IP地址>:26502/run
```

In [ ]:
import asyncio

import kachaka_api
import nest_asyncio
import uvicorn
from fastapi import BackgroundTasks, FastAPI

app = FastAPI()
client = kachaka_api.aio.KachakaApiClient()
lock = asyncio.Lock()


async def send_command():
    await client.speak("カチャカです、よろしくね！")
    print("send_command finished.")


async def run_task():
    if lock.locked():
        print("Another task is already running. Skip.")
        return
    async with lock:
        await send_command()


@app.get("/run")
async def handle_get_run(background_tasks: BackgroundTasks):
    background_tasks.add_task(run_task)
    return {"success": True}


nest_asyncio.apply()  # 在 JupyterLab 中執行 uvicorn 所需
uvicorn.run(app, host="0.0.0.0", port=26502)